In [ ]:
!pip install datasets wandb

In [1]:
import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, RandomSampler
from transformers import AutoTokenizer
from datasets import load_dataset
import os
import random
# import wandb

/home/ritish/prog/ML/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load the Multi30k dataset
dataset = load_dataset("bentrevett/multi30k", split="train")
val_dataset = load_dataset("bentrevett/multi30k", split="validation")
test_dataset = load_dataset("bentrevett/multi30k", split="test")

In [3]:
from transformers import BertTokenizer, DataCollatorForSeq2Seq

In [4]:
# Load tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
vocab_size = tokenizer.vocab_size

In [64]:
# Create a config for the model
config = {
    'n_embed': 256,
    'n_head': 8,
    'n_layer': 6,
    'block_size': 64,
    'dropout': 0.1,
    'batch_size': 64,
    'lr': 1,
    'tokenizer': 'bert-base-uncased',
}

In [ ]:
wandb.init(
    project='Transformer_multi30k_enc_final_layer',
    entity='ritishtest1',

    config=config
    )

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [65]:
# Tokenize the dataset
def tokenize_function(examples):
    source = tokenizer(examples["en"], padding="max_length", truncation=True, max_length=config['block_size'])
    target = tokenizer(examples["de"], padding="max_length", truncation=True, max_length=config['block_size'])
    # source = tokenizer(examples["en"], truncation=True, max_length=config['block_size'])
    # target = tokenizer(examples["de"], truncation=True, max_length=config['block_size'])
    source["labels"] = target["input_ids"]
    return source

In [66]:
tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=["en", "de"])
tokenized_val_datasets = val_dataset.map(tokenize_function, batched=True, remove_columns=["en", "de"])
tokenized_test_datasets = test_dataset.map(tokenize_function, batched=True, remove_columns=["en", "de"])

Map: 100%|██████████| 1000/1000 [00:00<00:00, 3436.75 examples/s]


In [67]:
# Create data collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=None)

# Create DataLoaders
train_dataloader = DataLoader(tokenized_datasets, shuffle=True, batch_size=config['batch_size'], collate_fn=data_collator)
val_dataloader = DataLoader(tokenized_val_datasets, shuffle=False, batch_size=config['batch_size'], collate_fn=data_collator)

In [68]:
class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size, causal, cross=False):
        super().__init__()
        self.num_heads = num_heads
        self.head_size = head_size
        self.causal = causal
        self.cross = cross
        self.q_attn = nn.Linear(config['n_embed'], config['n_embed'], bias=False)
        self.kv_attn = nn.Linear(config['n_embed'], config['n_embed'] * 2, bias=False)
        self.c_proj = nn.Linear(config['n_embed'], config['n_embed'], bias=False)             # Combine the outputs of the heads
        self.attn_dropout = nn.Dropout(config['dropout'])
        self.resid_dropout = nn.Dropout(config['dropout'])

        self.flash = hasattr(torch.nn.functional, 'scaled_dot_product_attention')

        ############################### REMOVE THIS #########################################
        self.flash = False

        if not self.flash:
            print("WARNING: using slow attention.")
            # self.register_buffer("tril", torch.tril(torch.ones(block_size, block_size)).view(1, 1, block_size, block_size))
            self.register_buffer("tril", torch.tril(torch.ones(config['block_size'], config['block_size'])))

    def forward(self, x, y=None, mask=None):
        B, T_dec, C = x.shape

        if y is None:
            y = x

        B, T_enc, C = y.shape

        # Query from the decoder
        q = self.q_attn(x).view(B, T_dec, self.num_heads, self.head_size).transpose(1, 2)  # (B,T_dec,C) -> (B,T_dec,nh,hs) -> (B,nh,T_dec,hs)
        k, v = self.kv_attn(y).split(C, dim=-1)                                 # (B,T_enc,2C) -> 2*(B,T_enc,C)
        k = k.view(B, T_enc, self.num_heads, self.head_size).transpose(1, 2)    # (B,nh,T_enc,hs)
        v = v.view(B, T_enc, self.num_heads, self.head_size).transpose(1, 2)    # (B,nh,T_enc,hs)

        if self.flash:
            out = torch.nn.functional.scaled_dot_product_attention(q, k, v, attn_mask=None, is_causal=self.causal)
        else:
            wei = q @ k.transpose(-2, -1) * 1.0 / math.sqrt(k.size(-1))         # (B,nh,T_dec,hs) @ (B,nh,hs,T_enc) -> (B,nh,T_dec,T_enc)
            if self.causal:
                if mask is None:
                    # Lower triangular mask to mask future tokens
                    wei = wei.masked_fill(self.tril[:T_dec, :T_enc] == 0, float('-inf')) # (B,nh,T_dec,T_enc)
                else:
                    wei = wei.masked_fill(mask == 0, float('-inf'))             # (B,nh,T_dec,T_enc)
            wei = F.softmax(wei, dim=-1)                                        # (B,nh,T_dec,T_enc)
            wei = self.attn_dropout(wei)
            out = wei @ v                                                       # (B,nh,T_dec,T_enc) @ (B,nh,T_enc,hs) -> (B,nh,T_dec,hs)

        out = out.transpose(1, 2).contiguous().view(B, T_dec, -1)               # (B,T_dec,nh*hs) = (B,T_dec,C)
        out = self.resid_dropout(self.c_proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embed):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embed, 4 * n_embed),
            nn.ReLU(),
            nn.Linear(4 * n_embed, n_embed),
            nn.Dropout(config['dropout']),
        )

    def forward(self, x):
        return self.net(x)

class DecoderBlock(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embed, n_head, cross=False):
        # n_embed: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        self.cross = cross
        head_size = n_embed // n_head
        self.sa = MultiHeadAttention(n_head, head_size, causal=True, cross=False)               # Masked MHA
        if self.cross:
            # Cross attention block
            self.cross_sa = MultiHeadAttention(n_head, head_size, causal=False, cross=True)     # Cross MHA
        self.ffwd = FeedFoward(n_embed)
        self.ln1 = nn.LayerNorm(n_embed)
        self.ln2 = nn.LayerNorm(n_embed)
        self.ln3 = nn.LayerNorm(n_embed)

    def forward(self, x, y=None):
        x = x + self.sa(self.ln1(x))
        if self.cross:
            x = x + self.cross_sa(self.ln2(x), y)
        x = x + self.ffwd(self.ln3(x))
        return x

class EncoderBlock(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embed, n_head):
        # n_embed: embedding dimension, nhead: the number of heads we'd like
        super().__init__()
        head_size = n_embed // n_head
        self.sa = MultiHeadAttention(n_head, head_size, causal=False, cross=False)
        self.ffwd = FeedFoward(n_embed)
        self.ln1 = nn.LayerNorm(n_embed)
        self.ln2 = nn.LayerNorm(n_embed)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# Encoder-decoder transformer
class Transformer(nn.Module):
    def __init__(self):
        super().__init__()
        # Create token embedding table
        self.token_embedding_table_enc = nn.Embedding(vocab_size, config['n_embed'])
        self.token_embedding_table_dec = nn.Embedding(vocab_size, config['n_embed'])

        # Create encoder blocks
        self.enc_blocks = nn.Sequential(*[EncoderBlock(config['n_embed'], config['n_head']) for _ in range(config['n_layer'])])

        # Create decoder blocks
        self.dec_blocks = nn.ModuleList([DecoderBlock(config['n_embed'], config['n_head'], cross=True) for _ in range(config['n_layer'])])

        self.ln_f = nn.LayerNorm(config['n_embed'])
        self.lm_head = nn.Linear(config['n_embed'], vocab_size)

    def generate_sinousidal_pos_embeddings(self, T, device):
        pos = torch.arange(T, device=device).view(-1,1)              # (T, 1)
        i = torch.arange(config['n_embed'], device=device).view(1,-1)          # (1, C)
        angles = pos / 10000**(2*i/(config['n_embed']))                        # (T, C)
        angles[:, 0::2] = torch.sin(angles[:, 0::2])
        angles[:, 1::2] = torch.cos(angles[:, 1::2])
        return angles.unsqueeze(0)                                   # (1,T,C)

    def forward(self, enc_inp, dec_inp, targets=None):
        B, T_enc = enc_inp.shape
        B, T_dec = dec_inp.shape

        # enc_inp and targets are both (B,T) tensor of integers
        tok_emb_enc = self.token_embedding_table_enc(enc_inp)                        # (B,T,C)
        tok_emb_dec = self.token_embedding_table_dec(dec_inp)                        # (B,T,C)

        # Encoder and decoder positional embeddings
        pos_emb_enc = self.generate_sinousidal_pos_embeddings(T_enc, device=enc_inp.device)  # (1,T_enc,C)
        pos_emb_dec = self.generate_sinousidal_pos_embeddings(T_dec, device=dec_inp.device)  # (1,T_dec,C)

        x_enc = tok_emb_enc + pos_emb_enc                                        # (B,T_enc,C)
        x_dec = tok_emb_dec + pos_emb_dec                                        # (B,T_dec,C)

        # Get output from encoder and decoder blocks
        x_enc_out = self.enc_blocks(x_enc)                                       # (B,T_enc,C)

        # Give output of the final encoder layer to the decoder layers
        for i in range(config['n_layer']):
            x_dec = self.dec_blocks[i](x_dec, x_enc_out)                         # (B,T_dec,C)

        x = self.ln_f(x_dec)                                                     # (B,T_dec,C)
        logits = self.lm_head(x)                                                 # (B,T_dec,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape

            logits = logits.reshape(B*T, C)
            targets = targets.reshape(B*T)
            loss = F.cross_entropy(logits, targets, ignore_index=0)

        return logits, loss

    def generate(self, enc_inp, max_len=config['block_size'], greedy=True):
        with torch.no_grad():
            dec_inp = 101 * torch.ones((1, 1), dtype=torch.long, device=enc_inp.device)

            for i in range(max_len-1):
                dec_inp = dec_inp[:, -config['block_size']:]
                logits, _ = self(enc_inp, dec_inp)
                if greedy:
                    next_token = torch.argmax(logits[:, -1, :], dim=-1).view(-1, 1)
                else:
                    next_token = torch.multinomial(F.softmax(logits[:, -1, :], dim=-1), num_samples=1).view(-1, 1)
                dec_inp = torch.cat([dec_inp, next_token], dim=1)
                if next_token == 102:
                    break

        return dec_inp

    def generate_v1(self, enc_inp, max_len=config['block_size'], greedy=True):
        with torch.no_grad():
            dec_inp = 101 * torch.ones((1, 1), dtype=torch.long, device=enc_inp.device)
            enc_inp = enc_inp[:, :config['block_size']]

            B, T_enc = enc_inp.shape
            x_enc = self.token_embedding_table_enc(enc_inp)
            pos_emb_enc = self.generate_sinousidal_pos_embeddings(T_enc, device=enc_inp.device)
            x_enc = x_enc + pos_emb_enc
            x_enc_out = self.enc_blocks(x_enc)

            for i in range(max_len-1):
                dec_inp = dec_inp[:, -config['block_size']:]
                x_dec = self.token_embedding_table_dec(dec_inp)
                pos_emb_dec = self.generate_sinousidal_pos_embeddings(dec_inp.shape[1], device=dec_inp.device)
                x_dec = x_dec + pos_emb_dec

                # Layer wise cross attention
                for j in range(config['n_layer']):
                    x_dec = self.dec_blocks[j](x_dec, x_enc_out)                        # (B,T_dec,C)

                x = self.ln_f(x_dec)                                                    # (B,T_dec,C)
                logits = self.lm_head(x)                                                # (B,T_dec,vocab_size)

                if greedy:
                    next_token = torch.argmax(logits[:, -1, :], dim=-1).view(-1, 1)
                else:
                    next_token = torch.multinomial(F.softmax(logits[:, -1, :], dim=-1), num_samples=1).view(-1, 1)
                dec_inp = torch.cat([dec_inp, next_token], dim=1)
                if next_token == 102:
                    break

        return dec_inp

In [69]:
model = Transformer()
print(sum(p.numel() for p in model.parameters())/1e6, 'M parameters')
n_epoch = 0
n_step = 0

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = 'cpu'
model = model.to(device)

34.512698 M parameters


In [70]:
# Learning rate scheduling with warmup steps
def lr_scheduler(step, n_embed=config['n_embed'], warmup_steps=400):
    step = max(1, step)
    lr = (n_embed ** -0.5) * min(step ** -0.5, step * warmup_steps ** -1.5)
    return lr

optimizer = torch.optim.AdamW(model.parameters(), lr=config['lr'])
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_scheduler)

In [91]:
# Create a new training loop that prints train and val loss at every 100 steps
# Training loop
max_epochs = 1
for epoch in range(max_epochs):
    n_epoch += 1
    model.train()
    total_loss = 0
    for step, batch in enumerate(train_dataloader):
        optimizer.zero_grad()
        input_ids = batch["input_ids"].to(device)
        labels = batch["labels"].to(device)
        labels[labels == -100] = 0
        decoder_input_ids = labels[:, :-1].contiguous()
        labels = labels.contiguous()[:, 1:]

        # Create labels for autoregressive training of decoder
        # decoder_input_ids = labels[:, :-1].contiguous()
        # labels = labels.contiguous()

        # # Add padding to the start of decoder input
        # decoder_input_ids = F.pad(decoder_input_ids, (1, 0), value=tokenizer.pad_token_id)

        # print(input_ids[0], input_ids[0].shape)
        # print(decoder_input_ids[0], decoder_input_ids[0].shape)
        # print(labels[0], labels[0].shape)
        # break

        # Forward pass
        logits, loss = model(input_ids, decoder_input_ids, labels)

        # Backward pass
        loss.backward()

        optimizer.step()
        scheduler.step()
        n_step += 1
        total_loss += loss.item()
        if step % 100 == 0:
            # Calculate val loss
            val_loss = 0
            model.eval()

            # Disable gradient calculation
            with torch.no_grad():
                for val_step, val_batch in enumerate(val_dataloader):
                    input_ids = val_batch["input_ids"].to(device)
                    labels = val_batch["labels"].to(device)

                    # Create labels for autoregressive training of decoder
                    decoder_input_ids = labels[:, :-1].contiguous()
                    labels = labels.contiguous()

                    # Add padding to the start of decoder input
                    decoder_input_ids = F.pad(decoder_input_ids, (1, 0), value=tokenizer.pad_token_id)

                    # Forward pass
                    logits, loss = model(input_ids, decoder_input_ids, labels)
                    val_loss += loss.item()

            for param_group in optimizer.param_groups:
                lr_log = param_group['lr']

            print(f"Epoch {epoch}, Step {step}, Train Loss: {total_loss/(step+1)}, Val Loss: {val_loss/val_step}, Learning Rate: {lr_log}")
            # wandb.log({"train_loss": total_loss/(step+1), "val_loss": val_loss/val_step, "learning_rate": lr_log, "step": n_step})
            model.train()

Epoch 0, Step 0, Train Loss: 1.6372112035751343, Val Loss: 2.4653658072153726, Learning Rate: 0.002072994146270811
Epoch 0, Step 100, Train Loss: 1.513190752208823, Val Loss: 2.4538716157277425, Learning Rate: 0.001967589218814246
Epoch 0, Step 200, Train Loss: 1.5102685108706726, Val Loss: 2.453014787038167, Learning Rate: 0.0018767837923068882
Epoch 0, Step 300, Train Loss: 1.4947394251427382, Val Loss: 2.36102131207784, Learning Rate: 0.0017974915891970336
Epoch 0, Step 400, Train Loss: 1.4836679019238288, Val Loss: 2.299164891242981, Learning Rate: 0.001727468727571194


In [97]:
model.eval()

# Generate translations
sample = val_dataset[np.random.randint(len(val_dataset))]
input_ids = tokenizer(sample["en"], padding='max_length', max_length=config['block_size'], truncation=True, return_tensors="pt").input_ids.to(device)
dec_inp = model.generate_v1(input_ids, max_len=100, greedy=True)

translation = tokenizer.decode(dec_inp[0], skip_special_tokens=True)
print("Source:", sample["en"])
print("Target:", sample["de"])
print("Translation:", translation)

Source: A soccer game is played as two men attempt to reach the ball before their respected opponent.
Target: Während eines Fußballspiels versuchen zwei Männer, den Ball vor ihrem jeweiligen Gegner zu erreichen.
Translation: ein fußballspieler bereitet sich auf den ball vor, den ball zum versuch, den ball zu fangen.


In [ ]:
model.eval()

input_ids = tokenizer.encode("Many people standing by a fountain underneath a blue and white umbrella.", return_tensors="pt", max_length=config['block_size'], truncation=True, padding='max_length').to(device)
dec_inp = model.generate_v1(input_ids, greedy=True)
print(tokenizer.decode(dec_inp[0], skip_special_tokens=True))

viele personen stehen am strand, um eine blaue schirmen und blaue schirmen.


In [ ]:
wandb.finish()

learning_rate,▁▃▅▆██▇▆▆▆▆▅▅▅▅▅▅▅▄▄▄▄▄▄▄▄▄▄▄▄▄
step,▁▁▂▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇███
train_loss,█▅▄▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
learning_rate,0.0012
step,2725
train_loss,0.74025
val_loss,1.44041
